## Decision Tree & Random Forest Models

Rather than handling the data as a series of time series, in these models I create basic features from the time series data for baseline performance.

In [99]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import duckdb
import re, gc

### Functions

In [100]:
# Create the rowwise sum of n days of observations for the specified range of columns; this range includes BOTH the start and end columns.
def n_day_sum(df, n=0, start_col=0, end_col=0, name_prefix=''):
    try:
        if (end_col - (start_col-1))%n != 0:
            raise ValueError('Column range must be evenly divisible by n.')
    except ValueError as e:
        print(f'{e}; DataFrame Unchanged; invalid column range. Start & end columns are included in range.')
        return df

    col_index = 0
    col_range = end_col - (start_col-1)
    cols_added = 0
    while col_index < col_range:
        sum_columns = df.iloc[:, start_col+col_index-1:start_col+col_index+n-1]
        sum = sum_columns.sum(axis=1)
        df[f'{name_prefix}nSum{n}_{cols_added}'] = sum
        col_index += n
        cols_added += 1

    return df

In [101]:
# Create the rowwise average of n days of observations for the specified range of columns; this range includes BOTH the start and end columns.
# Note: if you do this after using n_day_sum(), you will lose your sum columns and be left only with means. I'll fix it eventually.
# In the meantime, if you need both sums and means, do means first and then sums.
def n_day_mean(df, n=0, start_col=0, end_col=0, name_prefix=''):
    try:
        if (end_col - (start_col-1))%n != 0:
            raise ValueError('Column range must be evenly divisible by n.')
    except ValueError as e:
        print(f'{e}; DataFrame Unchanged; invalid column range. Start & end columns are included in range.')

    df = n_day_sum(df, n=n, start_col=start_col, end_col=end_col, name_prefix=name_prefix)
    
    cols_avgd = 0
    num_sums = (end_col - (start_col-1))/n
    while cols_avgd < num_sums:
        sum_col_name = f'{name_prefix}nSum{n}_{cols_avgd}'
        mean_col_name = f'{name_prefix}nMean{n}_{cols_avgd}'
        df[sum_col_name] = df[sum_col_name] / n
        df.rename(columns={sum_col_name: mean_col_name}, inplace=True)
        cols_avgd += 1

    return df

In [102]:
# Create the global yearly average & attach it to the dataframe.
def yearly_means(df):
    n_day_mean(df, n=240, start_col=6, end_col=245)
    query = duckdb.query('SELECT year, AVG(nMean240_0) FROM pr_m GROUP BY year ORDER BY year;').to_df()
    df.merge(query, on='year')
    return df

### DataFrame Preparation

#### Training set:

In [112]:
pr_m = pd.read_parquet('/Users/james/FutureCrop_Kaggle/Data/pr_maize_train.parquet')

In [113]:
# Create 30-day measures of precipitation
pr_m = n_day_mean(pr_m, n=30, start_col=6, end_col=245)
pr_m = n_day_sum(pr_m, n=30, start_col=6, end_col=245)
pr_m = yearly_means(pr_m)

In [114]:
sc_m = pd.read_parquet('/Users/james/FutureCrop_Kaggle/Data/soil_co2_maize_train.parquet')
sc_m.reset_index(inplace = True)
cols_to_select = ['ID', 'texture_class', 'co2', 'nitrogen']
sc_m = sc_m[cols_to_select]

In [115]:
cols_to_select = ['crop', 'year', 'lon', 'lat',
                  'nMean30_0', 'nMean30_1', 'nMean30_2', 'nMean30_3', 'nMean30_4', 'nMean30_5', 'nMean30_6', 'nMean30_7',
                 'nSum30_0', 'nSum30_1', 'nSum30_2', 'nSum30_3', 'nSum30_4', 'nSum30_5', 'nSum30_6', 'nSum30_7']

train_df = pr_m[cols_to_select]
train_df.reset_index(inplace = True)
train_df = train_df.merge(sc_m, on=['ID'])

In [116]:
ts_m = pd.read_parquet('/Users/james/FutureCrop_Kaggle/Data/train_solutions_maize.parquet')
ts_m.reset_index(inplace = True)
train_df = train_df.merge(ts_m, on=['ID'])

In [117]:
del pr_m, sc_m, ts_m
gc.collect()

185

In [120]:
train_df = pd.get_dummies(train_df, columns=['crop'])
# Need to make ID the index again

In [121]:
train_df.head()

,ID,year,lon,lat,nMean30_0,nMean30_1,nMean30_2,nMean30_3,nMean30_4,nMean30_5,nMean30_6,nMean30_7,nSum30_0,nSum30_1,nSum30_2,nSum30_3,nSum30_4,nSum30_5,nSum30_6,nSum30_7,texture_class,co2,nitrogen,yield,crop_maize
0,0,381.0,-122.25,48.25,0.000052,0.000039,0.000021,1.345042e-05,0.000006,0.000029,0.000077,0.000110,0.001551,0.001160,0.000631,0.000404,0.000173,0.000861,0.002306,0.003308,9.0,340.79,186.110992,5.595,True
1,1,381.0,-122.25,48.75,0.000053,0.000051,0.000027,1.764786e-05,0.000006,0.000042,0.000082,0.000109,0.001575,0.001523,0.000813,0.000529,0.000177,0.001264,0.002475,0.003283,9.0,340.79,186.110992,5.895,True
2,2,381.0,-122.25,49.25,0.000099,0.000083,0.000047,2.562528e-05,0.000011,0.000074,0.000143,0.000183,0.002972,0.002490,0.001422,0.000769,0.000337,0.002231,0.004295,0.005476,9.0,340.79,184.934006,3.023,True
3,3,381.0,-116.75,43.25,0.000006,0.000012,0.000009,8.881294e-08,0.000004,0.000026,0.000028,0.000044,0.000167,0.000348,0.000281,0.000003,0.000116,0.000779,0.000837,0.001331,9.0,340.79,186.110992,2.071,True
4,4,381.0,-116.75,43.75,0.000003,0.000011,0.000006,0.000000e+00,0.000004,0.000026,0.000030,0.000043,0.000092,0.000316,0.000183,0.000000,0.000109,0.000775,0.000892,0.001282,10.0,340.79,186.110992,2.239,True


#### Test set:

In [138]:
pr_m = pd.read_parquet('/Users/james/FutureCrop_Kaggle/Data/pr_maize_test.parquet')
pr_m = n_day_mean(pr_m, n=30, start_col=6, end_col=245)
pr_m = n_day_sum(pr_m, n=30, start_col=6, end_col=245)
pr_m = yearly_means(pr_m)

In [140]:
sc_m = pd.read_parquet('/Users/james/FutureCrop_Kaggle/Data/soil_co2_maize_test.parquet')
sc_m.reset_index(inplace = True)
cols_to_select = ['ID', 'texture_class', 'co2', 'nitrogen']
sc_m = sc_m[cols_to_select]

,ID,texture_class,co2,nitrogen
0,349719,9.0,418.06,186.110992
1,349720,9.0,418.06,186.110992
2,349721,9.0,418.06,184.934006
3,349722,9.0,418.06,186.110992
4,349723,9.0,418.06,186.110992


In [141]:
cols_to_select = ['crop', 'year', 'lon', 'lat',
                  'nMean30_0', 'nMean30_1', 'nMean30_2', 'nMean30_3', 'nMean30_4', 'nMean30_5', 'nMean30_6', 'nMean30_7',
                 'nSum30_0', 'nSum30_1', 'nSum30_2', 'nSum30_3', 'nSum30_4', 'nSum30_5', 'nSum30_6', 'nSum30_7']

test_df = pr_m[cols_to_select]
test_df.reset_index(inplace = True)
test_df = test_df.merge(sc_m, on=['ID'])

,ID,crop,year,lon,lat,nMean30_0,nMean30_1,nMean30_2,nMean30_3,nMean30_4,nMean30_5,nMean30_6,nMean30_7,nSum30_0,nSum30_1,nSum30_2,nSum30_3,nSum30_4,nSum30_5,nSum30_6,nSum30_7,texture_class,co2,nitrogen
0,349719,maize,420.0,-122.25,48.25,0.000027,0.000012,0.000006,0.000015,2.786551e-06,0.000052,0.000056,0.000065,0.000821,0.000362,0.000187,0.000439,0.000084,0.001556,0.001687,0.001963,9.0,418.06,186.110992
1,349720,maize,420.0,-122.25,48.75,0.000031,0.000012,0.000007,0.000016,3.054733e-06,0.000051,0.000066,0.000072,0.000942,0.000368,0.000205,0.000485,0.000092,0.001530,0.001984,0.002161,9.0,418.06,186.110992
2,349721,maize,420.0,-122.25,49.25,0.000056,0.000025,0.000017,0.000029,8.828045e-06,0.000094,0.000122,0.000120,0.001674,0.000742,0.000513,0.000885,0.000265,0.002807,0.003671,0.003607,9.0,418.06,184.934006
3,349722,maize,420.0,-119.75,47.75,0.000005,0.000004,0.000003,0.000006,2.145821e-06,0.000012,0.000002,0.000021,0.000149,0.000113,0.000077,0.000179,0.000064,0.000360,0.000062,0.000640,9.0,418.06,186.110992
4,349723,maize,420.0,-116.75,43.25,0.000019,0.000014,0.000002,0.000001,6.854866e-07,0.000023,0.000006,0.000031,0.000567,0.000407,0.000064,0.000042,0.000021,0.000699,0.000195,0.000923,9.0,418.06,186.110992


In [142]:
del pr_m, sc_m
gc.collect()

0

In [144]:
test_df = pd.get_dummies(test_df, columns=['crop'])
# need to make ID the index again

,ID,year,lon,lat,nMean30_0,nMean30_1,nMean30_2,nMean30_3,nMean30_4,nMean30_5,nMean30_6,nMean30_7,nSum30_0,nSum30_1,nSum30_2,nSum30_3,nSum30_4,nSum30_5,nSum30_6,nSum30_7,texture_class,co2,nitrogen,crop_maize
0,349719,420.0,-122.25,48.25,0.000027,0.000012,0.000006,0.000015,2.786551e-06,0.000052,0.000056,0.000065,0.000821,0.000362,0.000187,0.000439,0.000084,0.001556,0.001687,0.001963,9.0,418.06,186.110992,True
1,349720,420.0,-122.25,48.75,0.000031,0.000012,0.000007,0.000016,3.054733e-06,0.000051,0.000066,0.000072,0.000942,0.000368,0.000205,0.000485,0.000092,0.001530,0.001984,0.002161,9.0,418.06,186.110992,True
2,349721,420.0,-122.25,49.25,0.000056,0.000025,0.000017,0.000029,8.828045e-06,0.000094,0.000122,0.000120,0.001674,0.000742,0.000513,0.000885,0.000265,0.002807,0.003671,0.003607,9.0,418.06,184.934006,True
3,349722,420.0,-119.75,47.75,0.000005,0.000004,0.000003,0.000006,2.145821e-06,0.000012,0.000002,0.000021,0.000149,0.000113,0.000077,0.000179,0.000064,0.000360,0.000062,0.000640,9.0,418.06,186.110992,True
4,349723,420.0,-116.75,43.25,0.000019,0.000014,0.000002,0.000001,6.854866e-07,0.000023,0.000006,0.000031,0.000567,0.000407,0.000064,0.000042,0.000021,0.000699,0.000195,0.000923,9.0,418.06,186.110992,True


In [ ]:
test_df.head()

### Model Setup

In [ ]:
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
X_train = train_df.drop(labels=['ID','yield'], axis=1)
y_train = train_df['yield']

# Make sure columns line up
X_train, X_test = X_train.align(test_df, join='left', axis=1)

In [ ]:
d_tree = DecisionTreeRegressor()


In [ ]:
rand_f = RandomForestRegressor()